In [1]:
import requests
import os
from langchain.docstore.document import Document
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
import re

from dotenv import load_dotenv
load_dotenv()

base_url = os.environ["SERVICENOW_BASEURL"]
user = os.environ["SERVICENOW_USERNAME"]
pwd = os.environ["SERVICENOW_PASSWORD"]
headers = {"Content-Type":"application/json","Accept":"application/json"}


def get_specific_catalog_item(sys_id):

    url = f"{base_url}/sn_sc/servicecatalog/items/{sys_id}"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()
    data = response.json()
    print(data)
    return data




def get_table_response(reference):
    url = f"{base_url}/now/table/{reference}?sysparm_limit=1"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()
    data = response.json()
    return data


def get_table_sysid(reference, table_field, value_for_query):
    if("@" in value_for_query):
        value_for_query = value_for_query.replace("@","%40")
    sysparm_query = f"{table_field}%3D{value_for_query}" 

    url = f"{base_url}/now/table/{reference}?sysparm_query={sysparm_query}&sysparm_limit=1"
    # print(url)
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()

    data = response.json()
    return data

def get_table_basedon_ref_qualifier(reference, sys_param_query):
    url = f'{base_url}/now/table/{reference}?sysparm_query={sys_param_query}'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    data = response.json()
    print(data)


def get_catalog_item_variables(sys_id):
    url = f"{base_url}/sn_sc/servicecatalog/items/{sys_id}/variables"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
    print(response)
    data = response.json()
    return data

def get_table_values(reference):
    url = f"{base_url}/now/table/{reference}"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()
    data = response.json()
    return data

def get_tableValue_via_link(link):
    url = link
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()
    data = response.json()
    return data



In [2]:
from langchain.schema import HumanMessage
import json
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 


model = AzureChatOpenAI(
    api_key="107c20f6b6774c80b98c6f6a828f6374",
    azure_deployment="gpt-4",
    azure_endpoint="https://2000081253-openai.openai.azure.com/",
    api_version="2023-03-15-preview"
)

In [3]:
catalog_item_description = """

### Catalog Item Name: Report IT Incidents

#### Description:
This catalog item allows users to report IT incidents they are experiencing. It provides a structured form to capture all necessary details about the incident to ensure timely and accurate resolution by the IT support team.

#### Variables (Fields):

1. **Reported For**: This is a mandatory field where you can specify who is reporting the incident. It is a reference to the `sys_user` table and is pre-populated with the current user's details.

2. **Location**: This mandatory field allows you to specify the location related to the incident. It is a reference to the `cmn_location` table and dynamically pulls the location based on the `Reported For` field.    

3. **Contact Number**: This mandatory field is for providing a contact number where you can be reached for further information or updates about the incident.

4. **Project**: This mandatory field allows you to specify the project associated with the incident. It references the `cmn_department` table and dynamically derives the department based on the `Reported For` field.  

5. **Building**: This mandatory field is for specifying the building where the incident occurred.

6. **Service**: This mandatory field allows you to specify the service related to the incident. It references the `u_service` table with a qualifier to only show services where the `u_parent_field` is empty.

   **Choices**:
   - CPU
   - Firmware
   - Slow
   - Hunt Group Call Landing
   - Service Operations
   - Email Services
   - SLAs
   - Module
   - Microphone issue
   - Wireless Controller
   - Kramer Room scheduler not working
   - Switch / Router
   - Recording
   - Finesse
   - Attendance Tracker
   - Virus Infection
   - Desktop
   - Slowness issue
   - Connectivity issue
   - Error
   - User account disable - Batch job failure
   - Server or Cloud Alerts
   - Hardware
   - SFTP (Secure File Transfer Protocol)
   - Clouds VM maintenance
   - Virtual Server - Windows
   - Windows Updates
   - Port
   - Active Sync
   - Application Issue
   - End user computing (Deskside) services
   - DC Ops Services
   - Inbound Call
   - Hex_EY VPN Related Issues
   - Clearance Process
   - Links
   - Outbound Call
   - IVR (Interactive Voice Response)
   - Skype / Teams
   - Other Accessories
   - InactiveTerminalSessions
   - Break - Fix
   - User Account Management
   - IT Proof
   - VC room
   - Other Applications
   - Patch Management
   - IP Phone
   - Cloud Ops
   - Down
   - Audio issue
   - Customer Delight
   - Performance
   - PeopleSoft HRMS
   - Others
   - Other Software
   - Disk Clean Up
   - Software
   - Stationh Admin
   - VPN Related Issues
   - Asset Allocation Issues
   - Voice Instance
   - Touch panel issue
   - Database
   - Expense Claim System
   - CUIC
   - Webex
   - Printer / Scanner
   - Data Card
   - Data
   - Memory
   - Time & Billing
   - Antivirus Updates
   - Physical Server - Linux
   - ART
   - Operating System
   - Computer Accessories
   - Mobile App
   - Peripheral Devices
   - Password Reset
   - RMG Portal
   - Call Flow
   - Email
   - Service down
   - Plan Arena
   - Voice Services
   - Video Conferencing
   - Switch
   - Network Services
   - Performance Issue
   - Physical Server - Windows
   - Content sharing issue
   - SPAM Email
   - e-Voice
   - CPU Utilization
   - Printer
   - Mobile Device
   - Citrix
   - Other
   - Verint
   - Allowances
   - Video Conference
   - Mouse
   - MY HR
   - Visa Requisition
   - EPayroll
   - ION
   - Diskspace Utilization
   - Browser
   - Webex Login Issue
   - Storage
   - Global Protect VPN
   - MS Outlook
   - Backup / Restore
   - Compass
   - Mail Delivery Issue
   - CPPM
   - Router
   - Virtualization
   - User name / Email ID changes for existing
   - Alerts Monitoring
   - DID Mapping
   - Internet Connectivity
   - Audio Quality issues in WebEx Meeting
   - Firewall
   - Memory Utilization
   - Application Services
   - Virtual Server - Linux
   - WiFi
   - People Soft Finance
   - Scanner
   - VPN
   - Azure/Aws/GCP ticket creation
   - Server Management
   - Reporting
   - File Transfer
   - Camera issue
   - Reports
   - Application
   - Network alerts
   - SVN
   - Agent Login
   - Access
   - Account Unlock
   - Anti Virus
   - Vendor Payment System
   - Board Room
   - NetArch 9
   - DAT update
   - CRM
   - Revenue Tracker
   - IIS Operations Support
   - Microsoft Office
   - Full and Final Settlement
   - AskGenie
   - Keyboard
   - Login Issues
   - Nice
   - Access Related issue
   - Voice Quality
   - Laptop
   - AskGenie Administration
   - Service Request
   - Application or Database Alerts
   - Availability
   - NIC (Network Interface Card)
   - Unable to find Recordings or Deleted recordings
   - Agent
   - MyBenefits

7. **Category**: This mandatory field allows you to categorize the incident further. It references the `u_service` table and has a qualifier to dynamically show categories based on the selected service.

   **Choices**:
   - CPU
   - Firmware
   - Slow
   - Hunt Group Call Landing
   - Service Operations
   - Email Services
   - SLAs
   - Module
   - Microphone issue
   - Wireless Controller
   - Kramer Room scheduler not working
   - Switch / Router
   - Recording
   - Finesse
   - Attendance Tracker
   - Virus Infection
   - Desktop
   - Slowness issue
   - Connectivity issue
   - Error
   - User account disable - Batch job failure
   - Server or Cloud Alerts
   - Hardware
   - SFTP (Secure File Transfer Protocol)
   - Clouds VM maintenance
   - Virtual Server - Windows
   - Windows Updates
   - Port
   - Active Sync
   - Application Issue
   - End user computing (Deskside) services
   - DC Ops Services
   - Inbound Call
   - Hex_EY VPN Related Issues
   - Clearance Process
   - Links
   - Outbound Call
   - IVR (Interactive Voice Response)
   - Skype / Teams
   - Other Accessories
   - InactiveTerminalSessions
   - Break - Fix
   - User Account Management
   - IT Proof
   - VC room
   - Other Applications
   - Patch Management
   - IP Phone
   - Cloud Ops
   - Down
   - Audio issue
   - Customer Delight
   - Performance
   - PeopleSoft HRMS
   - Others
   - Other Software
   - Disk Clean Up
   - Software
   - Stationh Admin
   - VPN Related Issues
   - Asset Allocation Issues
   - Voice Instance
   - Touch panel issue
   - Database
   - Expense Claim System
   - CUIC
   - Webex
   - Printer / Scanner
   - Data Card
   - Data
   - Memory
   - Time & Billing
   - Antivirus Updates
   - Physical Server - Linux
   - ART
   - Operating System
   - Computer Accessories
   - Mobile App
   - Peripheral Devices
   - Password Reset
   - RMG Portal
   - Call Flow
   - Email
   - Service down
   - Plan Arena
   - Voice Services
   - Video Conferencing
   - Switch
   - Network Services
   - Performance Issue
   - Physical Server - Windows
   - Content sharing issue
   - SPAM Email
   - e-Voice
   - CPU Utilization
   - Printer
   - Mobile Device
   - Citrix
   - Other
   - Verint
   - Allowances
   - Video Conference
   - Mouse
   - MY HR
   - Visa Requisition
   - EPayroll
   - ION
   - Diskspace Utilization
   - Browser
   - Webex Login Issue
   - Storage
   - Global Protect VPN
   - MS Outlook
   - Backup / Restore
   - Compass
   - Mail Delivery Issue
   - CPPM
   - Router
   - Virtualization
   - User name / Email ID changes for existing
   - Alerts Monitoring
   - DID Mapping
   - Internet Connectivity
   - Audio Quality issues in WebEx Meeting
   - Firewall
   - Memory Utilization
   - Application Services
   - Virtual Server - Linux
   - WiFi
   - People Soft Finance
   - Scanner
   - VPN
   - Azure/Aws/GCP ticket creation
   - Server Management
   - Reporting
   - File Transfer
   - Camera issue
   - Reports
   - Application
   - Network alerts
   - SVN
   - Agent Login
   - Access
   - Account Unlock
   - Anti Virus
   - Vendor Payment System
   - Board Room
   - NetArch 9
   - DAT update
   - CRM
   - Revenue Tracker
   - IIS Operations Support
   - Microsoft Office
   - Full and Final Settlement
   - AskGenie
   - Keyboard
   - Login Issues
   - Nice
   - Access Related issue
   - Voice Quality
   - Laptop
   - AskGenie Administration
   - Service Request
   - Application or Database Alerts
   - Availability
   - NIC (Network Interface Card)
   - Unable to find Recordings or Deleted recordings
   - Agent
   - MyBenefits

8. **Child Category 1**: This mandatory field allows you to specify a sub-category related to the incident. It references the `u_service` table and has a qualifier to dynamically show sub-categories based on the selected category.

   **Choices**:
   - CPU
   - Firmware
   - Slow
   - Hunt Group Call Landing
   - Service Operations
   - Email Services
   - SLAs
   - Module
   - Microphone issue
   - Wireless Controller
   - Kramer Room scheduler not working
   - Switch / Router
   - Recording
   - Finesse
   - Attendance Tracker
   - Virus Infection
   - Desktop
   - Slowness issue
   - Connectivity issue
   - Error
   - User account disable - Batch job failure
   - Server or Cloud Alerts
   - Hardware
   - SFTP (Secure File Transfer Protocol)
   - Clouds VM maintenance
   - Virtual Server - Windows
   - Windows Updates
   - Port
   - Active Sync
   - Application Issue
   - End user computing (Deskside) services
   - DC Ops Services
   - Inbound Call
   - Hex_EY VPN Related Issues
   - Clearance Process
   - Links
   - Outbound Call
   - IVR (Interactive Voice Response)
   - Skype / Teams
   - Other Accessories
   - InactiveTerminalSessions
   - Break - Fix
   - User Account Management
   - IT Proof
   - VC room
   - Other Applications
   - Patch Management
   - IP Phone
   - Cloud Ops
   - Down
   - Audio issue
   - Customer Delight
   - Performance
   - PeopleSoft HRMS
   - Others
   - Other Software
   - Disk Clean Up
   - Software
   - Stationh Admin
   - VPN Related Issues
   - Asset Allocation Issues
   - Voice Instance
   - Touch panel issue
   - Database
   - Expense Claim System
   - CUIC
   - Webex
   - Printer / Scanner
   - Data Card
   - Data
   - Memory
   - Time & Billing
   - Antivirus Updates
   - Physical Server - Linux
   - ART
   - Operating System
   - Computer Accessories
   - Mobile App
   - Peripheral Devices
   - Password Reset
   - RMG Portal
   - Call Flow
   - Email
   - Service down
   - Plan Arena
   - Voice Services
   - Video Conferencing
   - Switch
   - Network Services
   - Performance Issue
   - Physical Server - Windows
   - Content sharing issue
   - SPAM Email
   - e-Voice
   - CPU Utilization
   - Printer
   - Mobile Device
   - Citrix
   - Other
   - Verint
   - Allowances
   - Video Conference
   - Mouse
   - MY HR
   - Visa Requisition
   - EPayroll
   - ION
   - Diskspace Utilization
   - Browser
   - Webex Login Issue
   - Storage
   - Global Protect VPN
   - MS Outlook
   - Backup / Restore
   - Compass
   - Mail Delivery Issue
   - CPPM
   - Router
   - Virtualization
   - User name / Email ID changes for existing
   - Alerts Monitoring
   - DID Mapping
   - Internet Connectivity
   - Audio Quality issues in WebEx Meeting
   - Firewall
   - Memory Utilization
   - Application Services
   - Virtual Server - Linux
   - WiFi
   - People Soft Finance
   - Scanner
   - VPN
   - Azure/Aws/GCP ticket creation
   - Server Management
   - Reporting
   - File Transfer
   - Camera issue
   - Reports
   - Application
   - Network alerts
   - SVN
   - Agent Login
   - Access
   - Account Unlock
   - Anti Virus
   - Vendor Payment System
   - Board Room
   - NetArch 9
   - DAT update
   - CRM
   - Revenue Tracker
   - IIS Operations Support
   - Microsoft Office
   - Full and Final Settlement
   - AskGenie
   - Keyboard
   - Login Issues
   - Nice
   - Access Related issue
   - Voice Quality
   - Laptop
   - AskGenie Administration
   - Service Request
   - Application or Database Alerts
   - Availability
   - NIC (Network Interface Card)
   - Unable to find Recordings or Deleted recordings
   - Agent
   - MyBenefits

9. **Short Description**: This mandatory field allows you to provide a brief summary of the incident.        

10. **Detailed Description**: This mandatory field allows you to provide a detailed description of the incident, including any relevant information that can help in resolving the issue.

11. **Add Attachment**: This optional field allows you to attach files related to the incident, with a maximum file size of 20 MB.


"""

In [4]:
def fetchVariables(data, variables_name):

    template = """{data}
    
    from the given data Return only the variables name as a key and variable description as a value in JSON.
    The description must describe contains choices and ui_policy too as a single output
    The variablename should be one of the name from the given List. {variables_name}
    And no premable or explaination.

    Example:
    variablename : variabledescription

    """
    
    prompt_template = PromptTemplate(template=template, input_variables=["data", variables_name])
    chain = prompt_template | model | JsonOutputParser()
    response = chain.invoke({"data": data, "variables_name": variables_name})
    print(response)
    return response

In [5]:
def create_custom_function(variable_info, custom_functions, variables_description):
    inner_json = {}
    inner_json["type"] = "string"
    if(variable_info["name"] in variables_description):
        inner_json["description"] = variables_description[variable_info["name"]]
        custom_functions[0]["parameters"]["properties"][variable_info["name"]] = inner_json
    return custom_functions


def function_calling_catVar(catalog_variables, catalog_item_description, variables_name):
    custom_functions = [
        {
            "name": "extract_catalog_variables",
            "description": "Get the values from the body of the user query",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
        ]
    variables_description = fetchVariables(catalog_item_description, variables_name)


    for variable_info in catalog_variables:
        if "children" in variable_info:
            for child_var_info in variable_info["children"]:
                # print(child_var_info)
                custom_functions = create_custom_function(child_var_info, custom_functions, variables_description)
        else:
            custom_functions = create_custom_function(variable_info, custom_functions, variables_description)
    return custom_functions


In [6]:
def get_all_variables_List(catalog_item_variable):
    def extract_variables(variables):
        variable_List = []
        for variable_info in variables:
            if variable_info["type"] == 24 or variable_info['type'] == 11:
                continue
            if "children" in variable_info:
                variable_List.extend(extract_variables(variable_info["children"]))
            else:
                variable_List.append(variable_info)
        return variable_List
    return extract_variables(catalog_item_variable)


def set_reference_value(variable):
    
    if(variable["type"] == 8 and variable["dynamic_value_field"] == ""):
        # if('ref_qualifier' in variable and variable['ref_qualifier'] != ''):
        #     ref_qualifier = variable['ref_qualifier']
        #     if(not ref_qualifier.startswith('javascript:new')):
        #         reference = variable["reference"]
        #         table_content = get_table_basedon_ref_qualifier(reference, ref_qualifier)
        #     else:
        #         pass
        # else:
        #     reference = variable["reference"]
        #     table_content = get_table_values(reference=reference)
        
        reference = variable["reference"]
        table_content = get_table_values(reference=reference)

        names_in_table = []
        for content in table_content["result"]:
            if("u_name" in content):
                names_in_table.append(content["u_name"])
        variable["reference_values"] = ", ".join(names_in_table)
        return variable
    
    elif (variable["type"] == 5 or variable["type"] == 18):
        choiceList = []
        for choice in variable["choices"]:
            choiceList.append(choice["value"])
        variable["reference_values"] = ", ".join(choiceList)
        return variable

    elif(variable["value"] != ""):
        variable["displayvalue"] = variable["value"]
        return variable

    else:
        return variable

def fetch_ui_actions(action_field, catalog_variables):
    ui_action_variable_list = []
    for action_variables in action_field:
        action_variable_id = action_variables["name"].split(":")[1] 
        for variable in catalog_variables:
            # # # print(f"{variable["id"]} = {action_variable_id}")
            if(variable["id"] == action_variable_id):
                ui_action_variable_list.append({"name": variable["name"], "mandatory": action_variables["mandatory"]})
    
    return ui_action_variable_list


def fetch_ui_policy(catalog_item, catalog_variables):
    final_ui_condition_action = []
    for catalog_ui_policy in catalog_item["result"]["ui_policy"]:
        for condition_field in catalog_ui_policy["conditions"]:
            action_field = catalog_ui_policy["actions"]
            ui_action_variable_list = fetch_ui_actions(action_field, catalog_variables)
            variable_id = condition_field["field"].split(":")[1]
            for variable in catalog_variables:
                if variable["id"] == variable_id:
                    ui_condition = {"condition_variable_name":variable["name"],"condition_variable_value": condition_field["value"], "condition_operation": condition_field["oper"], "ui_actions": ui_action_variable_list}
                    final_ui_condition_action.append(ui_condition)
    return final_ui_condition_action


def arrange_api_request(parse_variable_details,variables_template):
    for key in parse_variable_details:
        if key in variables_template:
            variables_template[key] = parse_variable_details[key]
    return variables_template


def get_valuefor_reqfor(variable_info, variables_template, fetched_variables):
        reference = variable_info["reference"]
        sysparm_for_query = "user_name"
        value_for_query = os.environ["SERVICENOW_USERNAME"]
        # print(reference + sysparm_for_query + value_for_query)
        sys_id = get_table_sysid(reference, sysparm_for_query, value_for_query)["result"][0]["sys_id"]
        variables_template[variable_info["name"]] = sys_id
        fetched_variables[variable_info["name"]] = sys_id
        return fetched_variables, variables_template


def get_valuefor_reference(variable_info, variables_template, fetched_variables, dynamic_value_dot_walk_path):
    reference = variable_info["reference"] # u_software

    sample_table_response = get_table_response(reference)
    if "u_name" in sample_table_response["result"][0]:
        sysparm_for_query = "u_name" # name
    elif "name" in sample_table_response["result"][0]:
        sysparm_for_query = "name"

    if(variable_info["name"] in fetched_variables):
        value_for_query = fetched_variables[variable_info["name"]] # python
        sys_id = get_table_sysid(reference, sysparm_for_query, value_for_query)["result"][0][dynamic_value_dot_walk_path]
        variables_template[variable_info["name"]] = sys_id
        
    return fetched_variables, variables_template



def is_valid_sys_id(value):
    return bool(re.fullmatch(r"[0-9a-fA-F]{32}", value))


def get_dynamicvalues_name(link):
    table_content = get_tableValue_via_link(link)
    print(table_content)
    value = ""
    if("name" in table_content['result']):
        value = table_content['result']['name']
    elif("u_name" in table_content):
        value = table_content["result"]['u_name']
    return value




def get_valuefor_dynamicvalues(variables_List, variable_info, variables_template, fetched_variables):

    for variables in variables_List:
        if(variables["id"] == variable_info["dynamic_value_field"]):
            reference = variables["reference"]
            sysparm_for_query = "sys_id"# variable_info["dynamic_value_dot_walk_path"]
            value_for_query = variables_template[variables["name"]]
        
            # print(" reference =  "+ reference +" sysparm_for_query =  "+ sysparm_for_query +" value_for_query = "+ value_for_query)
            # print([variable_info["dynamic_value_dot_walk_path"]])
            requireddetails = get_table_sysid(reference, sysparm_for_query, value_for_query)
            if("result" in requireddetails):
                resultValue = requireddetails["result"]
                if(isinstance(resultValue, list)):
                    print(resultValue)
                    if(0<len(resultValue)):
                        requireddetails = resultValue[0][variable_info["dynamic_value_dot_walk_path"]]
            finalDetail = ""
            # print(requireddetails)
            if("value" in requireddetails):
                finalDetail = requireddetails["value"]
                fetched_variables[variable_info["name"]] = get_dynamicvalues_name(requireddetails["link"])
            else:
                finalDetail = requireddetails
                fetched_variables[variable_info["name"]] = finalDetail
            variables_template[variable_info["name"]] = finalDetail

    print(fetched_variables)

    return fetched_variables, variables_template




def assign_complex_variables(fetched_variables, variables_template, variables_List):
    variables_template = arrange_api_request(fetched_variables, variables_template)
    for variable_info in variables_List:
        # print(variable_info["name"])
        if(variable_info["type"] == 31):            # for requested_for type
            fetched_variables, variables_template = get_valuefor_reqfor(variable_info, variables_template, fetched_variables)

        elif(variable_info["name"] in fetched_variables and variable_info["type"] == 8):
            fetched_variables, variables_template = get_valuefor_reference(variable_info, variables_template, fetched_variables, "sys_id")

        elif (variable_info["dynamic_value_field"] != ""):
            fetched_variables, variables_template = get_valuefor_dynamicvalues(variables_List, variable_info, variables_template, fetched_variables)
    return fetched_variables, variables_template

def fetch_mandatory_variables(variables_template, ui_policy,variables_List):
    missing_mandatory_variables = []
    ui_based_mandatory_variables = []
    for variable in variables_List:
        if(variable["mandatory"] == True):
            if(variables_template[variable["name"]] == ""):
                missing_mandatory_variables.append(variable["name"])
            ui_based_mandatory_variables.append(variable["name"])
    for ui_condition in ui_policy:
        if(ui_condition["condition_operation"] == "="):
            if(variables_template[ui_condition["condition_variable_name"]] == ui_condition["condition_variable_value"]):
                ui_actions = ui_condition["ui_actions"]
                for ui_action in ui_actions:
                    if(ui_action["mandatory"] == "true"):
                        if(variables_template[ui_action["name"]] == ""):
                            missing_mandatory_variables.append(ui_action["name"])
                        ui_based_mandatory_variables.append(ui_action["name"])
                    
    return variables_template, missing_mandatory_variables, ui_based_mandatory_variables

def prepare_api_request(user_query, sys_id, catalog_item_description):
    catalog_item = get_specific_catalog_item(sys_id)
    catalog_variables = catalog_item["result"]["variables"]
    ui_policy = fetch_ui_policy(catalog_item, get_all_variables_List(catalog_variables))
    variables_List = []
    variables_template = {}
    variables_name = []
    for variable in get_all_variables_List(catalog_variables):
        variable = set_reference_value(variable)
        variables_List.append(variable)
        variables_name.append(variable["name"])
        variables_template[variable["name"]] = variable["displayvalue"]
        
    custom_function = function_calling_catVar(variables_List, catalog_item_description, variables_name)
    message = model.predict_messages(
    [HumanMessage(content=f"user query: {user_query}")],
    functions = custom_function
    )

    print(variables_List)
    # print(custom_function)
    # print(json.dumps(message.__dict__))

    if message.additional_kwargs != {}:
        # print("The "additional_kwargs" attribute exists.")
        parse_variable_details = json.loads(message.additional_kwargs["function_call"]["arguments"])
        parse_variable_details,variables_template= assign_complex_variables(parse_variable_details,variables_template, variables_List)
        variables_template, missing_mandatory_variables, ui_based_mandatory_variables = fetch_mandatory_variables(variables_template, ui_policy,variables_List)
        
        final_parse_variable_details = {}
        print(f"parse variable : {parse_variable_details}")
        for variable in parse_variable_details:
            print(variable)
            if (variable in ui_based_mandatory_variables):
                final_parse_variable_details[variable] = parse_variable_details[variable]

    return catalog_item_description, final_parse_variable_details, variables_template, missing_mandatory_variables



In [7]:
prepare_api_request("raise a ticket for slow wifi speed in EB1 building", "39dbfcc3c3489a1068d8b132b4013167", catalog_item_description)

{'result': {'short_description': 'Get support with STG issues that you are facing', 'kb_article': '', 'icon': 'images/service_catalog/generic_small.gifx', 'description': '', 'availability': 'on_desktop', 'mandatory_attachment': False, 'request_method': '', 'type': 'catalog_item', 'visible_standalone': True, 'sys_class_name': 'sc_cat_item', 'sys_id': '39dbfcc3c3489a1068d8b132b4013167', 'content_type': '', 'order': 0, 'make_item_non_conversational': False, 'owner': '6816f79cc0a8016401c5a33be04be441', 'show_price': False, 'show_quantity': True, 'picture': '', 'url': '', 'catalogs': [{'sys_id': 'e0d08b13c3330100c8b837659bba8fb4', 'active': True, 'title': 'Service Catalog'}], 'name': 'Report IT Incidents', 'show_wishlist': True, 'category': {'sys_id': '', 'title': None}, 'turn_off_nowassist_conversation': False, 'show_delivery_time': True, 'categories': [], 'variables': [{'active': True, 'label': "Requestor's Information", 'dynamic_value_field': '', 'type': 11, 'mandatory': False, 'displayv

c:\Users\2000081253\Desktop\Work\AI-Copilot-Ticket-Assistant\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[{'active': True, 'label': 'Reported For', 'dynamic_value_field': '', 'type': 31, 'mandatory': True, 'displayvalue': 'javascript:gs.getUserID();', 'friendly_type': 'requested_for', 'display_type': 'Requested For', 'reference': 'sys_user', 'render_label': True, 'ref_qualifier': 'EQ', 'read_only': False, 'name': 'reported_for', 'attributes': 'edge_encryption_enabled=true', 'id': '6ddcfc07c3489a1068d8b132b401314d', 'value': 'javascript:gs.getUserID();', 'dynamic_value_dot_walk_path': '', 'help_text': '', 'max_length': 0, 'order': 1}, {'active': True, 'label': 'Location', 'dynamic_value_field': '6ddcfc07c3489a1068d8b132b401314d', 'type': 8, 'mandatory': True, 'displayvalue': '', 'friendly_type': 'reference', 'display_type': 'Reference', 'reference': 'cmn_location', 'render_label': True, 'ref_qualifier': '', 'read_only': False, 'name': 'location', 'attributes': 'edge_encryption_enabled=true', 'id': 'ba2d788bc3489a1068d8b132b4013182', 'value': '', 'dynamic_value_dot_walk_path': 'location', '

("\n\n### Catalog Item Name: Report IT Incidents\n\n#### Description:\nThis catalog item allows users to report IT incidents they are experiencing. It provides a structured form to capture all necessary details about the incident to ensure timely and accurate resolution by the IT support team.\n\n#### Variables (Fields):\n\n1. **Reported For**: This is a mandatory field where you can specify who is reporting the incident. It is a reference to the `sys_user` table and is pre-populated with the current user's details.\n\n2. **Location**: This mandatory field allows you to specify the location related to the incident. It is a reference to the `cmn_location` table and dynamically pulls the location based on the `Reported For` field.    \n\n3. **Contact Number**: This mandatory field is for providing a contact number where you can be reached for further information or updates about the incident.\n\n4. **Project**: This mandatory field allows you to specify the project associated with the in

In [8]:
parsed_data = {'business_service': 'Software Installation',
  'software_name': 'Python',
  'add_software': 'true',
  'software_name1': 'Java',
  'requested_for': '6816f79cc0a8016401c5a33be04be441'}

catalog_item = get_specific_catalog_item('50572ffac3405a1068d8b132b4013177')
catalog_variables = catalog_item["result"]["variables"]
ui_policy = fetch_ui_policy(catalog_item, get_all_variables_List(catalog_variables))
variables_List = []
variables_template = {}
variables_name = []
for variable in get_all_variables_List(catalog_variables):
    variable = set_reference_value(variable)
    variables_name.append(variable["name"])
    variables_template[variable["name"]] = variable["displayvalue"]
    for i in parsed_data:
        # print(i)
        if(variable['name'] == i):
            variable['value'] = parsed_data[i]
    variables_List.append(variable)

catalog_item["result"]["variables"] = variables_List
catalog_item["result"]["ui_policy"] = ui_policy

print(catalog_item)
print("")

{'result': {'short_description': 'Request for Software Services', 'kb_article': '', 'icon': 'images/service_catalog/generic_small.gifx', 'description': '<div>\r\n<ul><li>Please select Business Function/Request Type to display delivery time</li></ul>\r\n</div>', 'availability': 'on_desktop', 'mandatory_attachment': False, 'request_method': '', 'type': 'catalog_item', 'visible_standalone': True, 'sys_class_name': 'sc_cat_item', 'sys_id': '50572ffac3405a1068d8b132b4013177', 'content_type': '', 'order': 0, 'make_item_non_conversational': False, 'owner': '6816f79cc0a8016401c5a33be04be441', 'show_price': False, 'show_quantity': True, 'picture': '', 'url': '', 'catalogs': [{'sys_id': 'e0d08b13c3330100c8b837659bba8fb4', 'active': True, 'title': 'Service Catalog'}], 'name': 'Software Services', 'show_wishlist': True, 'category': {'sys_id': '01d72b3ec3405a1068d8b132b40131ac', 'title': 'Personal Catalogs'}, 'turn_off_nowassist_conversation': False, 'show_delivery_time': True, 'categories': [], 'v

In [9]:
catalog_item

{'result': {'short_description': 'Request for Software Services',
  'kb_article': '',
  'icon': 'images/service_catalog/generic_small.gifx',
  'description': '<div>\r\n<ul><li>Please select Business Function/Request Type to display delivery time</li></ul>\r\n</div>',
  'availability': 'on_desktop',
  'mandatory_attachment': False,
  'request_method': '',
  'type': 'catalog_item',
  'visible_standalone': True,
  'sys_class_name': 'sc_cat_item',
  'sys_id': '50572ffac3405a1068d8b132b4013177',
  'content_type': '',
  'order': 0,
  'make_item_non_conversational': False,
  'owner': '6816f79cc0a8016401c5a33be04be441',
  'show_price': False,
  'show_quantity': True,
  'picture': '',
  'url': '',
  'catalogs': [{'sys_id': 'e0d08b13c3330100c8b837659bba8fb4',
    'active': True,
    'title': 'Service Catalog'}],
  'name': 'Software Services',
  'show_wishlist': True,
  'category': {'sys_id': '01d72b3ec3405a1068d8b132b40131ac',
   'title': 'Personal Catalogs'},
  'turn_off_nowassist_conversation'

In [10]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key="107c20f6b6774c80b98c6f6a828f6374",  
    api_version="2023-03-15-preview",
    azure_endpoint = "https://2000081253-openai.openai.azure.com/"
    )
    
deployment_name='gpt-4' #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment. 
    

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

Yes, several other Azure AI services also support customer managed keys (CMK). Some of these services include:

1. **Azure Cognitive Services**
   - Azure Cognitive Services offers a variety of AI functionalities such as vision, speech, language, and decision services. These services support customer managed keys for greater control over your encryption keys.

2. **Azure Machine Learning**
   - Azure Machine Learning supports customer managed keys to help you maintain control over your data encryption at rest, providing an additional layer of security for your machine learning workflows and models.

3. **Azure Synapse Analytics**
   - Although not exclusively an AI service, Azure Synapse Analytics integrates with various AI and machine learning capabilities and also supports customer managed keys to protect your data.

These services provide the option to use your own encryption keys stored in Azure Key Vault, giving you greater control over key management and ensuring compliance with 

In [11]:
from langchain.schema import HumanMessage
import json
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 

model = AzureChatOpenAI(
    api_key="107c20f6b6774c80b98c6f6a828f6374",
    azure_deployment="gpt-4",
    azure_endpoint="https://2000081253-openai.openai.azure.com/",
    api_version="2023-03-15-preview"
)

data = "Tell me a joke"

template = """{data}"""

prompt_template = PromptTemplate(template=template, input_variables=["data"])
chain = prompt_template | model
response = chain.invoke({"data": data})
print(response)

content="Sure, here's a light-hearted one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!" response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 11, 'total_tokens': 33}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-34b8fc0a-e0d6-4364-bcbc-4478604d9a7e-0' usage_metadata={'input_tokens': 11, 'output_tokens': 22, 'total_tokens': 33}


In [12]:
def add_cart(sys_id, json_data):
    url = f'{base_url}/sn_sc/servicecatalog/items/{sys_id}/add_to_cart'
    response = requests.post(url, auth=(user, pwd), headers=headers ,data=json_data)
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    data = response.json()
    print(data)

In [13]:
def get_ref_qualifier_script(class_name):
    url = f'{base_url}/1353990/script_includes/script_include/{class_name}'

    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()

    data = response.json()
    print(data)

{'result': [{'sys_id': '4d70c587c3c89a1068d8b132b4013154', 'name': 'ShowCatagory', 'script': "var ShowCatagory = Class.create();\r\nShowCatagory.prototype = {\r\n    initialize: function() {},\r\n\r\n    catagory: function() {\r\n        var parent = current.variables.service;\r\n        var servicetable = new GlideRecord('u_service');\r\n        servicetable.addQuery('u_parent_field', parent);\r\n        servicetable.query();\r\n\r\n        parentfields = '';\r\n\r\n        while (servicetable.next()) {\r\n            parentfields += ',' + servicetable.sys_id;\r\n        }\r\n        return 'sys_idIN' + parentfields;\r\n    },\r\n\r\n    childCatagory: function() {\r\n        var parent = current.variables.category;\r\n        var servicetable = new GlideRecord('u_service');\r\n        servicetable.addQuery('u_parent_field', parent);\r\n        servicetable.query();\r\n\r\n        parentfields = '';\r\n\r\n        while (servicetable.next()) {\r\n            parentfields += ',' + serv